In [1]:
import os
import sys
import timeit
import pandas as pd
import numpy as np

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from eban import Eban

ModuleNotFoundError: No module named 'impala'

In [2]:
import os
import sys
import timeit
import pandas as pd
import numpy as np

# Loan

In [3]:
cohort = pd.date_range(start='2022-01-31',periods=10,freq='M')
cohort #billing date

DatetimeIndex(['2022-01-31', '2022-02-28', '2022-03-31', '2022-04-30',
               '2022-05-31', '2022-06-30', '2022-07-31', '2022-08-31',
               '2022-09-30', '2022-10-31'],
              dtype='datetime64[ns]', freq='M')

In [35]:
SQL_LOAN = """
with loan as (
select pos_dt , ar_id , ip_id,  pd_grp_tp_cd, pd_grp , opn_dt , cls_dt 
    , dsbr_dt , pymt_due_dt, nxt_pymt_dt, lmt_amt
    , dlq_dys, otsnd_bal, ptn_mm, ptn_yyyy
    , row_number() over (partition by ar_id order by ar_id, nxt_pymt_dt, pos_dt asc) as rn_loan
from datalake_view.vp_gnl_fnc_svc_ar 
where ptn_yyyy in (2021, 2022)
    and opn_dt > '2021-11-01'
    and spcl_prj_cd in ('ML6119','ML6118','ML6117','MCPG04','640','641','642'))


select base.nxt_pymt_dt as pos_dt
    , mls.ar_id
    , mls.ip_id
    , base.opn_dt
    , base.dsbr_dt
    , 'NULL' as pymy_due_dt
    , base.nxt_pymt_dt
    , base.lmt_amt
    , base.FPD_date as pos_dt_fpd
    , fpd.dlq_dys_fpd
    , fpd.otsnd_bal_fpd
    , base.enpl_date as pos_dt_enpl
    , enpl.dlq_dys_enpl
    , enpl.otsnd_bal_enpl
    , mls.fnl_aprv_tms
    , KWEB.brwr_brth_dt
    , KWEB.ocp_dsc
    , 'SE' as ocp_group
    , 'NULL' as tot_verf_incm_amt
    , KWEB.incm_amt
    , 'NULL' as fnl_cr_lmt
    , 'NULL' as fico_scor
    , seg.MODL_SEG


--KWEB
from (select pos_dt, ip_id, apl_id, brwr_brth_dt, incm_amt, ocp_dsc
    from DATATANK_VIEW.VP_DGTL_APL_LN_RGST) KWEB

--MLS
inner join (select IP_ID,nvl(apl_id,'0') as apl_id,max(pos_dt) as mx_dt,min(rgst_dt) as rgst_dt
from DATATANK_VIEW.VP_DGTL_APL_LN_RGST
where rgst_dt > '2021-11-01'
group by IP_ID ,nvl(apl_id,'0')) SL
on KWEB.IP_ID = SL.IP_ID
    and KWEB.POS_DT = SL.MX_DT
    and nvl(KWEB.apl_id,'0') = SL.apl_id
    
inner join (select M.ar_id, M.ip_id, M.ctr_id, M.pd_grp_tp_cd, M.aply_pd_cd, M.spcl_prj_cd
    , M.fnl_aprv_tms, m.fnl_lmt_amt
    from DATATANK_VIEW.VP_MLS_CTR_DTL M
inner join (select CTR_ID,max(pos_dt) as mx_dt
    from DATALAKE_VIEW.VP_MLS_CTR_DTL
    where ptn_yyyy = '2022'
        and spcl_prj_cd in ('ML6119','ML6118','ML6117','MCPG04','640','641','642')
    group by CTR_ID) C

where M.CTR_ID = C.CTR_ID
    and M.POS_DT = C.MX_DT
    and M.PTN_YYYY = '2022'
    and M.spcl_prj_cd in ('ML6119','ML6118','ML6117','MCPG04','640','641','642')
) MLS

on (KWEB.apl_id = MLS.CTR_ID)


--DPD
inner join (select ar_id , ip_id,  pd_grp_tp_cd, pd_grp , opn_dt , nxt_pymt_dt, lmt_amt,dsbr_dt, pymt_due_dt
    , date_add(nxt_pymt_dt,6) as FPD_date
    , date_add(nxt_pymt_dt,90) as enpl_date
    , ptn_mm, ptn_yyyy
    from loan
    where rn_loan = 1
        and nxt_pymt_dt = '{cohort}'
) as base
on base.ar_id = MLS.ar_id


left join (select pos_dt, ar_id
    , dlq_dys as dlq_dys_fpd, otsnd_bal as otsnd_bal_fpd
    from loan) as fpd
on fpd.ar_id = base.ar_id
    and fpd.pos_dt = base.FPD_date
    
left join (select pos_dt, ar_id
    , dlq_dys as dlq_dys_enpl , otsnd_bal as otsnd_bal_enpl
    from loan) as enpl
on enpl.ar_id = base.ar_id
    and enpl.pos_dt = base.enpl_date
    
LEFT JOIN (SELECT  IP_ID,MODL_SEG,PTN_YYYY,PTN_MM
    FROM FRDEPT_SANDBOX.V_ISCORE)SEG
ON cast(base.IP_ID as int) = SEG.IP_ID
    AND seg.PTN_YYYY = FROM_TIMESTAMP(base.OPN_DT,'yyyy')
    AND seg.PTN_MM =FROM_TIMESTAMP(base.OPN_DT,'MM')
    
order by mls.ar_id"""

In [36]:
for i in cohort:
    cohort_dt = i.strftime('%Y-%m-%d')
    print(cohort_dt)

    start_time = timeit.default_timer()

    FRDEPT_SANBOX = 'frdept_sandbox'
    eban = Eban(db_name=FRDEPT_SANBOX)
    eban.cursor.execute("SET request_pool = 'root.garage_pool';")
    eban.cursor.execute(SQL_LOAN.format(cohort=cohort_dt))
    running_time = str(timedelta(minutes=timeit.default_timer() - start_time))

    queried_data = eban.get_result()
    eban.close_db()
    print(f"Execution times: {running_time}")
    download_path = os.path.join(os.getcwd(),
                             'CSA ZIP',        # folder
                             #'LOAN_'+cohort_dt+'_test.xlsx')      # file name
                             'CSA_LOAN_'+cohort_dt+'.txt.gz')      # file name
    #queried_data.to_csv(download_path,sep='|', index=False)
    queried_data.to_csv(download_path,sep='|', index=False,compression='gzip')
    #queried_data.to_excel(download_path,index=False)
    queried_data.head(3)

2022-11-30
Execution times: 3:20:15.958423


### OD

In [26]:
cohort = pd.date_range(start='2022-03-01',periods=9,freq='MS')
cohort #open date

DatetimeIndex(['2022-03-01', '2022-04-01', '2022-05-01', '2022-06-01',
               '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01',
               '2022-11-01'],
              dtype='datetime64[ns]', freq='MS')

In [27]:
fpd_date = ['2022-04-05','2022-05-06','2022-06-06','2022-07-06','2022-08-05','2022-09-06','2022-10-06','2022-11-04','2022-12-06']
enpl_date = ['2022-06-29','2022-07-26','2022-08-29','2022-09-28','2022-10-28','2022-11-29','2022-12-29','2023-01-27','2023-02-28']

In [28]:
for i,j,k in zip(cohort,fpd_date,enpl_date):
    print('open date:',i,'FPD date:',j,'ENPL date:',k,"\n")

open date: 2022-03-01 00:00:00 FPD date: 2022-04-05 ENPL date: 2022-06-29 

open date: 2022-04-01 00:00:00 FPD date: 2022-05-06 ENPL date: 2022-07-26 

open date: 2022-05-01 00:00:00 FPD date: 2022-06-06 ENPL date: 2022-08-29 

open date: 2022-06-01 00:00:00 FPD date: 2022-07-06 ENPL date: 2022-09-28 

open date: 2022-07-01 00:00:00 FPD date: 2022-08-05 ENPL date: 2022-10-28 

open date: 2022-08-01 00:00:00 FPD date: 2022-09-06 ENPL date: 2022-11-29 

open date: 2022-09-01 00:00:00 FPD date: 2022-10-06 ENPL date: 2022-12-29 

open date: 2022-10-01 00:00:00 FPD date: 2022-11-04 ENPL date: 2023-01-27 

open date: 2022-11-01 00:00:00 FPD date: 2022-12-06 ENPL date: 2023-02-28 



In [8]:
SQL_OD = """
with od as(select *
from (select M.ar_id, M.opn_dt, M.tot_lmt_amt ,od_tp
    , row_number() over(partition by m.ar_id order by m.ar_id, m.pos_dt) as rn_od
    from datalake_view.V_PCB_AR_CA_DLY M
    join datalake_view.v_pcb_ar_ca_lmt_dly D
    on trunc(m.opn_dt,'MM') = '{cohort}' --first of month
        and (m.coa_ac = '1202001' or m.od_lmt_f = 'Y')
        and m.ar_id = d.ar_id
        and d.od_tp in ('640','641','642')
        and m.pos_dt = d.pos_dt) odod
where rn_od = 1)


select base.opn_dt as pos_dt
    , mls.ar_id
    , mls.ip_id
    , base.opn_dt
    , 'NULL' as dsbr_dt
    , 'NULL' as pymt_due_dt
    , 'NULL' as nxt_pymt_dt
    , base.tot_lmt_amt
    , base.FPD_date as pos_dt_fpd
    , fpd.dlq_dys_fpd
    , fpd.otsnd_bal_fpd
    , base.enpl_date as pos_dt_enpl
    , enpl.dlq_dys_enpl
    , enpl.otsnd_bal_enpl
    , mls.fnl_aprv_tms
    , KWEB.brwr_brth_dt
    , KWEB.ocp_dsc
    , 'SE' as ocp_group
    , 'NULL' as tot_verf_incm_amt
    , KWEB.incm_amt
    , 'NULL' as fnl_cr_lmt
    , 'NULL' as fico_scor
    , seg.MODL_SEG


--KWEB
from (select pos_dt, ip_id, apl_id, brwr_brth_dt, incm_amt, ocp_dsc
    from DATATANK_VIEW.VP_DGTL_APL_LN_RGST) KWEB

--MLS
inner join (select IP_ID,nvl(apl_id,'0') as apl_id,max(pos_dt) as mx_dt,min(rgst_dt) as rgst_dt
from DATATANK_VIEW.VP_DGTL_APL_LN_RGST
where rgst_dt > '2021-11-16'
group by IP_ID ,nvl(apl_id,'0')) SL
on KWEB.IP_ID = SL.IP_ID
    and KWEB.POS_DT = SL.MX_DT
    and nvl(KWEB.apl_id,'0') = SL.apl_id
    
inner join (select M.ar_id, M.ip_id, M.ctr_id, M.pd_grp_tp_cd, M.aply_pd_cd, M.spcl_prj_cd
    , M.fnl_aprv_tms, m.fnl_lmt_amt
    from DATATANK_VIEW.VP_MLS_CTR_DTL M
inner join (select CTR_ID,max(pos_dt) as mx_dt
    from DATALAKE_VIEW.VP_MLS_CTR_DTL
    where ptn_yyyy = '2022'
        and spcl_prj_cd in ('ML6119','ML6118','ML6117','MCPG04','640','641','642')
    group by CTR_ID) C

where M.CTR_ID = C.CTR_ID
    and M.POS_DT = C.MX_DT
    and M.PTN_YYYY = '2022'
    and M.spcl_prj_cd in ('ML6119','ML6118','ML6117','MCPG04','640','641','642')
) MLS

on (KWEB.apl_id = MLS.CTR_ID)


-- DPD
inner join (select ar_id, opn_dt, tot_lmt_amt
    , '{fpd_date}' as FPD_date
    , '{enpl_date}' as enpl_date
    from od) as base
on base.ar_id = MLS.ar_id


left join (select pos_dt, ar_id
    , dlq_dys as dlq_dys_fpd, otsnd_bal as otsnd_bal_fpd
    from datalake_view.VP_LPM_AR_INF 
    where pos_dt = '{fpd_date}'
    ) as fpd
on fpd.ar_id = base.ar_id
    
left join (select pos_dt, ar_id
    , dlq_dys as dlq_dys_enpl, otsnd_bal as otsnd_bal_enpl
    from datalake_view.VP_LPM_AR_INF 
    where pos_dt = '{enpl_date}') as enpl
on enpl.ar_id = base.ar_id
    
    
    
LEFT JOIN (SELECT  IP_ID,MODL_SEG,PTN_YYYY,PTN_MM
    FROM FRDEPT_SANDBOX.V_ISCORE)SEG
ON mls.ip_id = SEG.IP_ID
    AND seg.PTN_YYYY = FROM_TIMESTAMP(base.OPN_DT,'yyyy')
    AND seg.PTN_MM =FROM_TIMESTAMP(base.OPN_DT,'MM')
order by mls.ar_id"""

In [31]:
for i,j,k in zip(cohort,fpd_date,enpl_date):
    cohort_dt = i.strftime('%Y-%m-%d')
    fpd_dt = datetime.strptime(j,'%Y-%m-%d').strftime('%Y-%m-%d')
    enpl_dt = datetime.strptime(k,'%Y-%m-%d').strftime('%Y-%m-%d')
    print(cohort_dt)

    start_time = timeit.default_timer()

    FRDEPT_SANBOX = 'frdept_sandbox'
    eban = Eban(db_name=FRDEPT_SANBOX)
    eban.cursor.execute("SET request_pool = 'root.garage_pool';")
    eban.cursor.execute(SQL_OD.format(cohort=cohort_dt,fpd_date=fpd_dt,enpl_date=enpl_dt))
    running_time = str(timedelta(minutes=timeit.default_timer() - start_time))

    queried_data = eban.get_result()
    eban.close_db()
    print(f"Execution times: {running_time}")
    download_path = os.path.join(os.getcwd(),
                             'CSA OUTPUT NEW',        # folder
                             #'LOAN_'+cohort_dt+'_test.xlsx')      # file name
                             'CSA_OD_'+cohort_dt+'.txt.gz')      # file name
    #queried_data.to_csv(download_path,sep='|', index=False)
    queried_data.to_csv(download_path,sep='|', index=False,compression='gzip')
    #queried_data.to_excel(download_path,index=False)
    queried_data.head(3)

2022-10-01
Execution times: 2:31:40.820388
2022-11-01
Execution times: 1:56:36.071169


### _____________________________________

In [29]:
cohort = pd.date_range(start='2022-10-01',periods=2,freq='MS')
fpd_date = ['2022-11-04','2022-12-06']
enpl_date = ['2023-01-27','2023-02-28']

In [30]:
for i,j,k in zip(cohort,fpd_date,enpl_date):
    print('open date:',i,'FPD date:',j,'ENPL date:',k,"\n")

open date: 2022-10-01 00:00:00 FPD date: 2022-11-04 ENPL date: 2023-01-27 

open date: 2022-11-01 00:00:00 FPD date: 2022-12-06 ENPL date: 2023-02-28 



In [ ]:
#test sql script
for i,j,k in zip(cohort,fpd_date,enpl_date):
    cohort_dt = i.strftime('%Y-%m-%d')
    fpd_dt = datetime.strptime(j,'%Y-%m-%d').strftime('%Y-%m-%d')
    enpl_dt = datetime.strptime(k,'%Y-%m-%d').strftime('%Y-%m-%d')
    print(cohort_dt,'\n')
    print(SQL_OD.format(cohort=cohort_dt,fpd_date=fpd_dt,enpl_date=enpl_dt),'\n','-------------')